In [1]:
import tensorflow as tf

In [2]:
def cube(x):
    return x ** 3

In [3]:
#用python值调用
cube(2)

8

In [4]:
#用tensor调用
cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

#### tf.function()：将python函数转换为TF函数
可分析cube()函数执行的计算,并在后台生成等效的计算图

TF可以优化计算图、修剪未使用的结点、简化表达式等；TF函数通常比原始python函数运行的快，相当于增强了的原始python函数。

在 keras模型 中使用的 自定义函数 都会被keras自动转换为tf函数。可在 创建自定义层或模型时 设置dynamic=True 或 在调用模型编译compile（）时 设置run_eagerly=True，告诉keras不要将python函数转换为tf函数。

In [5]:
tf_cube = tf.function(cube)
tf_cube

In [6]:
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [7]:
tf_cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

#### 使用tf.function作为修饰器

In [8]:
@tf.function
def cube(x):
    return x ** 3

In [25]:
#限定参数的输入类型和输入的名称
#函数有了input_signature在TensorFlow中才可以保存这个model
@tf.function(input_signature=[tf.TensorSpec([None], tf.int32, name='x')]) 
def cube(x):
    return tf.pow(x, 3)

In [9]:
#使用原python函数
tf_cube.python_function(2)

8

默认时TF函数会为每个不同输入形状和数据类型生成一个新图形，并将其缓存以供后续使用。

当TF函数处理多态（变化的参数类型和形状）的张量参数时，相同形状相同数据类型的调用将重用一张计算图；但处理python值时将为每个不同值生成一个新图。

应尽量不用多个不同的python值调用它，python值应保留给很少有唯一值的参数。

#### get_concrete_function（）：使@tf.function标注的函数转成带有input_signature的函数,以此保存这个model

In [10]:
#指定输入张量的形状和类型就生成一个ConcreteFunction对象
concrete_function = tf_cube.get_concrete_function(tf.constant(2.0))
concrete_function.graph##查看图结构（funcGraph）

In [11]:
concrete_function(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [12]:
#看用新参数获得的对象与原来的对象是否一样
concrete_function is tf_cube.get_concrete_function(tf.constant(2.0))#图复用

True

#### 查看图结构

In [28]:
concrete_function.graph#图定义

#### 查看图所有的operations

In [14]:
ops = concrete_function.graph.get_operations()
ops

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'pow/y' type=Const>,
 <tf.Operation 'pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

#### 查看某个operation

In [21]:
pow_op = ops[2]
print(pow_op)

name: "pow"
op: "Pow"
input: "x"
input: "pow/y"
attr {
  key: "T"
  value {
    type: DT_FLOAT
  }
}



In [22]:
list(pow_op.inputs)

[<tf.Tensor 'x:0' shape=() dtype=float32>,
 <tf.Tensor 'pow/y:0' shape=() dtype=float32>]

In [23]:
list(pow_op.outputs)

[<tf.Tensor 'pow:0' shape=() dtype=float32>]

#### 通过operation的名字来查看类型(placeholder表示放输入的）

In [17]:
concrete_function.graph.get_operation_by_name('x')#查找name为‘x’的操作

<tf.Operation 'x' type=Placeholder>

#### 获得Identity的tensor，一般都要加 ：0

In [18]:
concrete_function.graph.get_tensor_by_name('Identity:0')#Identity是名字，0是索引

<tf.Tensor 'Identity:0' shape=() dtype=float32>

#### 查看函数特征

In [19]:
concrete_function.function_def.signature

name: "__inference_cube_16"
input_arg {
  name: "x"
  type: DT_FLOAT
}
output_arg {
  name: "identity"
  type: DT_FLOAT
}

#### 查看全部note

In [24]:
concrete_function.graph.as_graph_def()

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
      }
    }
  }
}
node {
  name: "pow/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
        }
        float_val: 3.0
      }
    }
  }
}
node {
  name: "pow"
  op: "Pow"
  input: "x"
  input: "pow/y"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "Identity"
  op: "Identity"
  input: "pow"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
versions {
  producer: 898
}

### 自动图和跟踪

In [29]:
@tf.function
def tf_cube(x):
    print("print:", x)
    return x ** 3

In [30]:
result = tf_cube(tf.constant(2.0))

print: Tensor("x:0", shape=(), dtype=float32)


In [31]:
result

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [32]:
result = tf_cube(2)
result = tf_cube(3)
result = tf_cube(tf.constant([[1., 2.]])) # New shape: trace!
result = tf_cube(tf.constant([[3., 4.], [5., 6.]])) # New shape: trace!

#多个不同的python数值参与了图的计算，引起了不必要的追踪，
#使得程序运行速度降低并消耗大量RAM
result = tf_cube(tf.constant([[7., 8.], [9., 10.], [11., 12.]])) # New shape: trace!

print: 2
print: 3
print: Tensor("x:0", shape=(1, 2), dtype=float32)
print: Tensor("x:0", shape=(2, 2), dtype=float32)
print: Tensor("x:0", shape=(3, 2), dtype=float32)


In [33]:
##限定参数的输入类型和输入的名称
@tf.function(input_signature=[tf.TensorSpec([None, 28, 28], tf.float32)])
def shrink(images):
    print("Tracing", images)
    return images[:, ::2, ::2] #“::2”中None是索引，2是步长

In [34]:
img_batch_1 = tf.random.uniform(shape=[100, 28, 28])
img_batch_2 = tf.random.uniform(shape=[50, 28, 28])
preprocessed_images = shrink(img_batch_1) # Traces the function.生成图
preprocessed_images = shrink(img_batch_2) # Reuses the same concrete function.重用图

Tracing Tensor("images:0", shape=(None, 28, 28), dtype=float32)


In [35]:
#当输入类型与input_signature不匹配时报错
img_batch_3 = tf.random.uniform(shape=[2, 2, 2])
try:
    preprocessed_images = shrink(img_batch_3)  # rejects unexpected types or shapes
except ValueError as ex:
    print(ex)

Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor(
[[[0.24022925 0.7350352 ]
  [0.66188097 0.25364745]]

 [[0.56562996 0.2113316 ]
  [0.06154072 0.91091084]]], shape=(2, 2, 2), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name=None)).


#### 调用任何外部库时仅在跟踪过程中运行，不会成为图表的一部分

In [39]:
import numpy as np

@tf.function
def f(x):
    return np.random.rand()

In [41]:
result = f(tf.constant(2.))
print(result)

tf.Tensor(0.29717714, shape=(), dtype=float32)


仅在跟踪函数时，即产生新的计算图时才会生成随机数

In [42]:
result = f(tf.constant(3.))#图复用，返回相同随机数
print(result)

tf.Tensor(0.29717714, shape=(), dtype=float32)


In [43]:
result = f(tf.constant([2.,3.]))#返回新的随机数
print(result)

tf.Tensor(0.8901182, shape=(), dtype=float32)
